<h1>Decision Tree Regression - Financial Stock Price Indication</h1>

<h3>Danayal Khan</h3>

<h4>Importing Libraries</h4>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import graphviz
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import ShuffleSplit

from sklearn.model_selection import GridSearchCV



Reading data 

In [2]:
df = pd.read_csv(r'C:\Users\danay\OneDrive\Desktop\AUS\Spring20\MachineLearning\FinancialDataProject\200-financial-indicators-of-us-stocks-20142018\2014_Financial_Data.csv', index_col = "Abbrv")
df

,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,Earnings before Tax,...,Receivables growth,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,Sector,2015 PRICE VAR [%],Class
Abbrv,,,,,,,,,,,,,,,,,,,,,
PG,7.440100e+10,-0.0713,3.903000e+10,3.537100e+10,0.0,2.146100e+10,2.146100e+10,1.391000e+10,7.090000e+08,1.449400e+10,...,-0.0187,-0.0217,0.0359,0.0316,0.1228,0.0000,-0.1746,Consumer Defensive,-9.323276,0
VIPS,3.734148e+09,1.1737,2.805625e+09,9.285226e+08,108330319.7,3.441414e+08,7.939267e+08,1.345959e+08,1.214869e+07,1.753823e+08,...,NaN,NaN,NaN,NaN,NaN,1.6484,1.7313,Consumer Defensive,-25.512193,0
KR,9.837500e+10,0.0182,7.813800e+10,2.023700e+10,0.0,1.519600e+10,1.751200e+10,2.725000e+09,4.430000e+08,2.270000e+09,...,0.0618,0.0981,0.1886,0.3268,0.2738,0.0000,0.0234,Consumer Defensive,33.118297,1
RAD,2.552641e+10,0.0053,1.820268e+10,7.323734e+09,0.0,6.561162e+09,6.586482e+09,7.372520e+08,4.245910e+08,2.502180e+08,...,0.0211,-0.0510,-0.0189,0.1963,-0.0458,0.0000,-0.0060,Consumer Defensive,2.752291,1
GIS,1.790960e+10,0.0076,1.153980e+10,6.369800e+09,0.0,3.474300e+09,3.412400e+09,2.957400e+09,3.024000e+08,2.707700e+09,...,0.0257,0.0090,0.0215,0.0274,0.1025,0.0000,-0.0220,Consumer Defensive,12.897715,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TSRI,4.952987e+07,0.1028,4.125164e+07,8.278229e+06,0.0,8.253061e+06,8.253061e+06,2.516800e+04,0.000000e+00,-6.893600e+04,...,-0.0384,0.0000,-0.0041,-0.0049,0.0000,0.0000,0.0213,Technology,29.362884,1
TZOO,1.532400e+08,-0.1019,1.917400e+07,1.340660e+08,11326000.0,1.125130e+08,1.162560e+08,1.781000e+07,0.000000e+00,1.790100e+07,...,0.1529,0.0000,-0.1872,0.1823,0.0000,0.2830,-0.0637,Technology,-31.167763,0
USATP,4.200000e+07,NaN,2.700000e+07,1.500000e+07,NaN,1.400000e+07,1.500000e+07,0.000000e+00,0.000000e+00,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Technology,-23.558900,0


Applying classifying function to generate the classes that will be used as labels

In [3]:
def classifying(x):
    if x > 1: 
        if x < 10:
            return 1
        if x < 100:
            return 2
        if x < 1000:
            return 3
        if x < 10000:
            return 4
        if x < 100000:
            return 5
        if x > 1000000:
            return 6
    
    else:
        if x > -10:
            return -1
        if x > -100:
            return -2
        if x > -1000:
            return -3
        if x < 1000:
            return -4
        

In [4]:
df["classes"] = df['2015 PRICE VAR [%]'].apply(classifying)


Dropping other labels that allowed us to generate the final "classes" label

In [5]:
df = df.drop(columns=['2015 PRICE VAR [%]','Class', 'Sector'])

In [6]:
df = df.fillna(0)

In [7]:
# # creating initial dataframe
# class_types = ('Class A','Class B','Class C','Class D','Class E','Class F','Class W', 'Class X', 'Class Y', 'Class Z')
# class_df = pd.DataFrame(class_types, columns=['class_types'])
# # creating instance of labelencoder
# labelencoder = LabelEncoder()
# # Assigning numerical values and storing in another column
# class_df['class_types_Cat'] = labelencoder.fit_transform(class_df['class_types'])
# class_df

In [9]:
nparray = df.to_numpy()
nparray

array([[ 7.44010000e+10, -7.13000000e-02,  3.90300000e+10, ...,
         0.00000000e+00, -1.74600000e-01, -1.00000000e+00],
       [ 3.73414805e+09,  1.17370000e+00,  2.80562544e+09, ...,
         1.64840000e+00,  1.73130000e+00, -2.00000000e+00],
       [ 9.83750000e+10,  1.82000000e-02,  7.81380000e+10, ...,
         0.00000000e+00,  2.34000000e-02,  2.00000000e+00],
       ...,
       [ 4.20000000e+07,  0.00000000e+00,  2.70000000e+07, ...,
         0.00000000e+00,  0.00000000e+00, -2.00000000e+00],
       [ 3.40758000e+08,  1.34400000e-01,  3.15948000e+08, ...,
         0.00000000e+00,  6.50000000e-02,  1.00000000e+00],
       [ 4.03373680e+07,  1.92500000e-01,  2.12936750e+07, ...,
         2.77800000e-01, -2.68000000e-02, -2.00000000e+00]])

Splitting the data into features and labels

In [10]:
features = nparray[:,0:-1]
label = nparray[:,-1]

(3808, 221)
[-1. -2.  2. ... -2.  1. -2.]


Splitting matrices into random train and test subsets n number of times to ensure there is no single random split bias

In [12]:
for i in range (100):
    X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=42)


In [13]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(3046, 221)
(762, 221)
(3046,)
(762,)


In [20]:
clf_1 = DecisionTreeClassifier(max_depth=1000, min_samples_split = 100, min_samples_leaf = 100, max_leaf_nodes = 990)


clf_1.fit(X_train, y_train)


# Predict
y_1 = clf_1.predict(X_test)


tree_para = {'criterion':['gini','entropy'],'max_leaf_nodes':[4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150], 'max_depth':[5,10,15,20,30,40,50,60]}
clf_4 = GridSearchCV(DecisionTreeClassifier(), tree_para, cv=5)
clf_4.fit(X_train, y_train)

print('Predicion accuracy is: ', clf_1.score(X_test, y_test))

print('Predicion accuracy is: ', clf_4.score(X_test, y_test))



C:\Users\danay\Anaconda3\envs\deeplearning\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


Predicion accuracy is:  0.4540682414698163
Predicion accuracy is:  0.37926509186351703
Predicion accuracy is:  0.3779527559055118
Predicion accuracy is:  0.463254593175853


Apply decision tree (DT) classifier on the dataset and determine the training and testing accuracy. 

Choose the best tree size by tweaking the Decision Tree hyperparameters. A plot showing the DT complexity vs. the training and testing accuracy

